###  
Handwritten character Recogniser

In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

In [11]:
import os
import cv2
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

# Define dataset path
dataset_path = "/kaggle/input/hand-dataset"  # Ensure this path is correct
IMG_SIZE = 32  # Image resize dimensions

# Get class names and create label mapping
class_names = sorted(os.listdir(dataset_path))  # Sorted for consistency
LABEL_MAP = {name: idx for idx, name in enumerate(class_names)}

print(f"✅ Found {len(class_names)} classes: {class_names}")

# Custom dataset class
class HandwritingDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        img_path = self.image_paths[index]
        label = self.labels[index]

        # Read image in grayscale
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            raise ValueError(f"⚠️ Error loading image: {img_path}")

        # Resize and normalize
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        img = np.expand_dims(img, axis=0)  # Add channel dimension (1, H, W)
        img = img.astype(np.float32) / 255.0  # Normalize

        if self.transform:
            img = self.transform(img)

        return torch.tensor(img, dtype=torch.float32), torch.tensor(label, dtype=torch.long)



✅ Found 62 classes: ['0', '1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '4', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '5', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '6', '60', '61', '7', '8', '9']


In [12]:
# Load dataset
image_paths = []
labels = []

for class_name in tqdm(class_names, desc="Loading Images"):
    class_path = os.path.join(dataset_path, class_name)
    if not os.path.isdir(class_path):
        continue

    for img_name in os.listdir(class_path):
        img_path = os.path.join(class_path, img_name)
        image_paths.append(img_path)
        labels.append(LABEL_MAP[class_name])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(image_paths, labels, test_size=0.2, random_state=42, stratify=labels)

# Transformations
transform = transforms.Compose([
    transforms.ToTensor()
])

Loading Images: 100%|██████████| 62/62 [00:00<00:00, 723.39it/s]


In [14]:
# Create PyTorch datasets
train_dataset = HandwritingDataset(X_train, y_train, transform=transform)
test_dataset = HandwritingDataset(X_test, y_test, transform=transform)

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(f"✅ Dataset Loaded: Training Samples={len(train_dataset)}, Testing Samples={len(test_dataset)}")


✅ Dataset Loaded: Training Samples=2728, Testing Samples=682


In [4]:
# Function to calculate accuracy
def calculate_accuracy(model, dataloader, criterion, device):
    model.eval()  # Set model to evaluation mode
    correct = 0
    total = 0
    total_loss = 0.0

    with torch.no_grad():  # No gradient calculation for validation
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            _, predicted = torch.max(outputs, 1)  # Get predicted class
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    accuracy = (correct / total) * 100
    avg_loss = total_loss / len(dataloader)
    return accuracy, avg_loss


# Training loop with accuracy & loss tracking
EPOCHS = 20
for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct_train += (predicted == labels).sum().item()
        total_train += labels.size(0)

    # Calculate training accuracy and average loss
    train_accuracy = (correct_train / total_train) * 100
    train_loss = running_loss / len(train_loader)

    # Calculate validation accuracy and loss
    val_accuracy, val_loss = calculate_accuracy(model, test_loader, criterion, device)

    print(f"Epoch [{epoch+1}/{EPOCHS}]")
    print(f"  🏋️‍♂️ Training  - Loss: {train_loss:.4f}, Accuracy: {train_accuracy:.2f}%")
    print(f"  📊 Validation - Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.2f}%\n")


Epoch [1/20]
  🏋️‍♂️ Training  - Loss: 0.6174, Accuracy: 78.52%
  📊 Validation - Loss: 0.8619, Accuracy: 73.75%

Epoch [2/20]
  🏋️‍♂️ Training  - Loss: 0.5793, Accuracy: 79.40%
  📊 Validation - Loss: 0.8857, Accuracy: 72.14%

Epoch [3/20]
  🏋️‍♂️ Training  - Loss: 0.5491, Accuracy: 79.95%
  📊 Validation - Loss: 0.8561, Accuracy: 74.49%

Epoch [4/20]
  🏋️‍♂️ Training  - Loss: 0.5237, Accuracy: 81.01%
  📊 Validation - Loss: 0.8887, Accuracy: 73.31%

Epoch [5/20]
  🏋️‍♂️ Training  - Loss: 0.5106, Accuracy: 81.60%
  📊 Validation - Loss: 0.8894, Accuracy: 73.75%

Epoch [6/20]
  🏋️‍♂️ Training  - Loss: 0.5001, Accuracy: 81.96%
  📊 Validation - Loss: 0.8986, Accuracy: 74.05%

Epoch [7/20]
  🏋️‍♂️ Training  - Loss: 0.5014, Accuracy: 82.62%
  📊 Validation - Loss: 0.9051, Accuracy: 73.31%

Epoch [8/20]
  🏋️‍♂️ Training  - Loss: 0.4592, Accuracy: 83.47%
  📊 Validation - Loss: 0.9319, Accuracy: 73.61%

Epoch [9/20]
  🏋️‍♂️ Training  - Loss: 0.4436, Accuracy: 84.05%
  📊 Validation - Loss: 0.9361, A

In [7]:
# Save model
torch.save(model.state_dict(), "handwritten_characters_model_recogniser.pth")
print("✅ Model training complete & saved!")


✅ Model training complete & saved!
